In [4]:
from langchain.chains import LLMRouterChain, LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE
from dotenv import load_dotenv
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router import MultiPromptChain
import os, warnings
warnings.filterwarnings('ignore')

In [6]:
load_dotenv()
api_key = os.environ['OPENAI_API_KEY']

In [8]:
# User asks a question about a location for travel
# INPUT -> ROUTER -> LLM decides which chain to direct towards -> Chain -> OUTPUT

In [90]:
travel_recommendations_template = """
You are a travel expert who specializes in providing recommendations for attractions, local experiences and must-visit places for a specific location.
Here is user's desired location:\n{input}
"""

In [92]:
visa_regulations_template = """
You are a visa and immigration expert who provides expert guidance on visa requirements, application processes and immigration policies for a specific location.
Here is user's desired location:\n{input}
"""

In [94]:
accommodations_template = """
You are a real estate expert who has enormous knowledge about all types of accommodations in a specific location.
Provide hotel recommendations, price comparisons and alternative stay options in {input}
"""

In [96]:
food_dining_template = """
You are a food and dining expert who has expert knowledge about all dishes and cuisines available within a specific location.
Suggest must-try foods, restaurant recommendations and dietary options in {input}
"""

In [98]:
travel_budget_template = """
You are a travel budget expert who has ample knowledge of total budget required for travel to a specific location.
Offer budget-planning trips and real-time currency exchange rates for travel to {input}
"""

In [100]:
# Route Prompt Information
# [] -> {} -> Name, Description and Template
prompt_info = [
    {
        "name": "destination guide",
        "description": "Recommends tourist attractions and must-visit places for a location",
        "template": travel_recommendations_template
    },
    {
        "name": "visa & immigration",
        "description": "Guides users on visa requirements and immigration policies for a location",
        "template": visa_regulations_template
    },
    {
        "name": "accommodation",
        "description": "Provides hotel recommendations and price comparisons for a location",
        "template": accommodations_template
    },
    {
        "name": "travel budget & currency",
        "description": "Offers budget planning tips and real-time currency exchange rates for a location",
        "template": travel_budget_template
    },
    {
        "name": "food & dining",
        "description": "Suggests must-try foods and restaurants for a location",
        "template": food_dining_template
    }
]

In [124]:
llm = ChatOpenAI(api_key=api_key)

In [128]:
destinations_chain = {}

for p_info in prompt_info:
    name = p_info['name']
    prompt_template = p_info['template']
    prompt = ChatPromptTemplate.from_template(prompt_template)
    chain = LLMChain(llm=llm,prompt=prompt)
    destinations_chain[name] = chain

In [130]:
default_template = "{input}"
default_prompt = ChatPromptTemplate.from_template(default_template)
default_chain = LLMChain(llm=llm,prompt=default_prompt)

In [132]:
print(MULTI_PROMPT_ROUTER_TEMPLATE)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (must include ```json at the start of the respon

In [134]:
destinations = [f"{p_info['name']}: {p_info['description']}" for p_info in prompt_info]
destinations

['destination guide: Recommends tourist attractions and must-visit places for a location',
 'visa & immigration: Guides users on visa requirements and immigration policies for a location',
 'accommodation: Provides hotel recommendations and price comparisons for a location',
 'travel budget & currency: Offers budget planning tips and real-time currency exchange rates for a location',
 'food & dining: Suggests must-try foods and restaurants for a location']

In [136]:
destinations_str = "\n".join(destinations)
print(destinations_str)

destination guide: Recommends tourist attractions and must-visit places for a location
visa & immigration: Guides users on visa requirements and immigration policies for a location
accommodation: Provides hotel recommendations and price comparisons for a location
travel budget & currency: Offers budget planning tips and real-time currency exchange rates for a location
food & dining: Suggests must-try foods and restaurants for a location


In [138]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)
print(router_template)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
destination guide: Recommends tourist attractions and must-visit places for a location
visa & immigra

In [140]:
router_prompt = PromptTemplate(template=router_template,
                               input_variables=['input'],
                               output_parser=RouterOutputParser())

In [142]:
router_chain = LLMRouterChain.from_llm(llm=llm,prompt=router_prompt)

In [144]:
chain = MultiPromptChain(router_chain=router_chain,
                         destination_chains=destinations_chain,
                         default_chain=default_chain,
                         verbose=True)

In [148]:
print(chain.run("What are the best places to visit in Barcelona?"))



> Entering new MultiPromptChain chain...
destination guide: {'input': 'What are the best places to visit in Barcelona?'}
> Finished chain.
Barcelona is a vibrant and diverse city with a plethora of attractions to explore. Here are some must-visit places in Barcelona:

1. Sagrada Familia - This iconic basilica designed by Antoni Gaudí is a must-see attraction in Barcelona. The breathtaking architecture and intricate details make it a truly unique and special place to visit.

2. Park Güell - Another masterpiece by Gaudí, Park Güell is a beautiful park filled with colorful mosaics, unique structures, and stunning views of the city. It's a great spot to relax and enjoy the beauty of Barcelona.

3. La Rambla - This bustling street in the heart of Barcelona is a popular spot for tourists and locals alike. Lined with shops, restaurants, and street performers, La Rambla is a great place to soak up the lively atmosphere of the city.

4. Gothic Quarter - Explore the narrow streets and historic

In [150]:
print(chain.run("What documents do I need to apply for a Japanese visa?"))



> Entering new MultiPromptChain chain...
visa & immigration: {'input': 'What documents do I need to apply for a Japanese visa?'}
> Finished chain.
To apply for a Japanese visa, you will typically need the following documents:

1. Passport: Your passport must be valid for at least six months beyond your intended stay in Japan.
2. Visa application form: You will need to fill out a visa application form, which can usually be downloaded from the website of the Japanese embassy or consulate in your country.
3. Passport-sized photo: You will need to provide a recent passport-sized photo of yourself.
4. Proof of travel arrangements: This can include a flight itinerary or reservation, hotel booking, or a letter of invitation from a host in Japan.
5. Proof of sufficient funds: You may need to provide bank statements or other financial documents to show that you have enough funds to cover your expenses during your stay in Japan.
6. Employment or income verification: If you are employed, you ma

In [152]:
print(chain.run("Are there any good hostels for backpackers in Berlin?"))



> Entering new MultiPromptChain chain...
accommodation: {'input': 'Are there any good hostels for backpackers in Berlin?'}
> Finished chain.
Yes, Berlin has a great selection of hostels for backpackers. Here are some recommendations:

1. Generator Hostel Berlin Mitte - This hostel is modern, clean, and centrally located. Prices start at around $25 per night for a bed in a dorm room.

2. Circus Hostel - Located in the trendy neighborhood of Prenzlauer Berg, Circus Hostel offers a fun and social atmosphere. Prices start at around $20 per night for a bed in a dorm room.

3. Wombats City Hostel Berlin - This hostel is known for its friendly staff and clean facilities. Prices start at around $25 per night for a bed in a dorm room.

In addition to hostels, budget travelers in Berlin can also consider alternative stay options such as Airbnb rentals, guesthouses, or budget hotels. Prices for these accommodations can vary, but they often provide more privacy and amenities compared to hostels.

In [154]:
print(chain.run("Are there any travel budget tips for a first-time visitor to Paris?"))



> Entering new MultiPromptChain chain...
travel budget & currency: {'input': 'Are there any travel budget tips for a first-time visitor to Paris?'}
> Finished chain.
Absolutely! Here are some travel budget tips for a first-time visitor to Paris:

1. Research and plan your itinerary beforehand to avoid last-minute expenses and maximize your time in the city.
2. Consider staying in a budget-friendly accommodation option such as a hostel, Airbnb, or budget hotel to save on lodging costs.
3. Take advantage of free or discounted attractions such as visiting parks, museums on free admission days, and exploring neighborhoods on foot.
4. Opt for public transportation like the metro or buses instead of taxis to save on transportation costs.
5. Eat like a local by trying out street food, visiting local markets, and dining at budget-friendly restaurants instead of touristy spots.
6. Purchase a Paris Pass or museum pass to save money on admissions to popular attractions.
7. Keep an eye out for d

In [156]:
print(chain.run("Where can I find authentic pasta in Rome?"))



> Entering new MultiPromptChain chain...
food & dining: {'input': 'Where can I find authentic pasta in Rome?'}
> Finished chain.
When in Rome, authentic pasta is a must-try! Here are some recommendations for where to find the best pasta in Rome:

1. Trattoria Da Luigi - This cozy trattoria in the heart of Rome is known for its homemade pasta dishes, particularly the cacio e pepe and amatriciana. The quaint atmosphere and friendly service make it a favorite among locals and tourists alike.

2. Roscioli - This renowned deli and restaurant offers a wide selection of pasta dishes made with fresh, high-quality ingredients. Their carbonara and truffle pasta are highly recommended. Be sure to also try their famous pizza bianca.

3. Felice a Testaccio - Located in the Testaccio neighborhood, this family-run trattoria has been serving up traditional Roman cuisine for generations. Their tonnarelli cacio e pepe is a standout dish that shouldn't be missed.

4. Il Pagliaccio - For a more upscale 